# Module import

In [1]:
# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import datetime, os
import time

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, transforms, build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode

# Dataloader

In [2]:
def get_cracks_dicts(img_dir):
    dataset_dicts = []
    image_paths = list(sorted(os.listdir(img_dir + 'image/')))
    mask_paths = None
    if os.path.exists(img_dir+'mask/'):
        mask_paths = list(sorted(os.listdir(img_dir+'mask/')))
    for idx, v in enumerate(image_paths):
        record = {}
        
        filename = os.path.join(img_dir, 'image/', v)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        if mask_paths:
            mask_path = img_dir + "mask/" + mask_paths[idx]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # HxWxC
            contours,_=cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

            objs = []
            for cnt in contours : 
                approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True) 
                if approx.shape[0] < 3:
                    continue
                # cv2.drawContours(image, [approx], 0, (0, 0, 255), 1) 
                # get bounding box coordinates for each mask
                px = [pos[0][0] for pos in approx]
                py = [pos[0][1] for pos in approx]
                poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                poly = [p for x in poly for p in x]
                obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
            record["sem_seg_file_name"] = mask_path # for segmentation evaluator
            record["seg_file_name"] = mask_path
        else:
            record["annotations"] = [{
                    "bbox": [0, 0, 0, 0],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [(0, 0), (0, 0), (0, 0)],
                    "category_id": 0,
                    "iscrowd": 0
                }]
            record["sem_seg_file_name"] = '' # for segmentation evaluator
        dataset_dicts.append(record)
    return dataset_dicts

# Setting for dataset and model

In [38]:
test_dir = '/home/aicenter/Documents/hsu/data/EWSHM/exp3_validation/'

In [39]:
DatasetCatalog.clear()
DatasetCatalog.register("cracks_test", lambda d= "test": get_cracks_dicts(test_dir))
MetadataCatalog.get("cracks_test").set(thing_classes=["crack"], thing_colors=[(96, 151, 255)])

namespace(name='cracks_test',
          thing_classes=['crack'],
          thing_colors=[(96, 151, 255)])

In [36]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (crack)
cfg.MODEL.WEIGHTS = "./models/EWSHM/exp3/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("cracks_test", )
predictor = DefaultPredictor(cfg)

## Get the number of parameters

In [28]:
model_parameters = filter(lambda p: p.requires_grad, predictor.model.backbone.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

45516800

## iterator for test dataset 

In [40]:
test_loader = build_detection_test_loader(cfg, "cracks_test")

# mIoU Evaluation

In [41]:
# borrows from pytorch-deeplab-xception
from utils.metrics import Evaluator
evaluator = Evaluator(2)

In [42]:
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])
    
evaluator.reset()
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    target = cv2.imread(d[0]['seg_file_name'], cv2.IMREAD_GRAYSCALE)
    evaluator.add_batch(target/255, mask)

MIoU = np.diag(evaluator.confusion_matrix) / (
    np.sum(evaluator.confusion_matrix, axis=1) + np.sum(evaluator.confusion_matrix, axis=0) -
    np.diag(evaluator.confusion_matrix))
Acc = evaluator.Pixel_Accuracy()
Acc_class = evaluator.Pixel_Accuracy_Class()
mIoU = evaluator.Mean_Intersection_over_Union()
FWIoU = evaluator.Frequency_Weighted_Intersection_over_Union()
print('Validation:')
print('mIoU:{}'.format(MIoU))
print("Acc:{}, Acc_class:{}, mIoU:{}, fwIoU: {}".format(Acc, Acc_class, mIoU, FWIoU))

/home/aicenter/Documents/hsu/data/EWSHM/exp3_validation/image/00021.png
/home/aicenter/Documents/hsu/data/EWSHM/exp3_validation/image/crop_93.png
Validation:
mIoU:[0.97280165 0.29616697]
Acc:0.9731094110479055, Acc_class:0.779738265870734, mIoU:0.6344843090506382, fwIoU: 0.9595667021788953


# DRAW ONLY SEGEMENTATION

In [43]:
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])

save_dir = test_dir + '/result/'
os.makedirs(save_dir, exist_ok=True)

mask_color = np.array((0, 255, 0), dtype="uint8")
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    roi = im[mask]
    im[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
    cv2.imwrite(save_dir + file_name.split('/')[-1], im)

/home/aicenter/Documents/hsu/data/EWSHM/exp3_validation/image/00021.png
/home/aicenter/Documents/hsu/data/EWSHM/exp3_validation/image/crop_93.png


# DRAW ALL

In [28]:
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])

save_dir = test_dir + '/result/'
os.makedirs(save_dir, exist_ok=True)

crack_metadata = MetadataCatalog.get("cracks_test")

for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
        
    v = Visualizer(im[:, :, ::-1],
                   metadata=crack_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # save_dir = './result/segmentation/'
    cv2.imwrite(save_dir + file_name.split('/')[-1], v.get_image()[:, :, ::-1])

/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_0.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_57.png


# CECI Data Visualization

### Mode 1: input with cropped image and do image stitching

In [8]:
crop_w, crop_h = 256, 256
ceci_dir = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/%sx%s/image/' %(crop_w, crop_h)
source_path = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/gt_640.png'

height, width, _ = cv2.imread(source_path).shape
x_num, y_num = int(width/crop_w), int(height/crop_h)

mask_color = np.array((0, 0, 255), dtype="uint8") 
whole_image = np.array([])
v_im = np.array([])
for filename in sorted(os.listdir(ceci_dir)):
    count_id = int(filename.split('.')[0])
    image_path = ceci_dir+filename
    im = cv2.imread(image_path)
    outputs = predictor(im)
    
    masks = outputs['instances'].pred_masks.cpu().numpy()
    # masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    masks = masks[outputs['instances'].scores.cpu().numpy() > 0.5]
    mask = np.sum(masks, axis=0) > 0
    roi = im[mask]
    im[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
    if (count_id+1)%y_num == 0:
        whole_image = v_im if whole_image.size == 0 else np.hstack((whole_image, v_im))
        v_im = np.array([])
    else:
        v_im = im if v_im.size == 0 else np.vstack((v_im, im))

save_path = source_path.split('.png')[0] + '_seg.png'
print('Saved at', save_path)
cv2.imwrite(save_path, whole_image)

Saved at /home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/gt_640_seg.png


True

### Mode 2: input of the whole picture

In [11]:
tunnel_image_path = '/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/3yi_tunnel.png' 
tunnel_image = cv2.imread(tunnel_image_path)
height, width, _ = tunnel_image.shape

crop_width, crop_height = (400, 400)
mask_color = np.array((0, 255, 0), dtype="uint8") 
x_num = int(width/crop_width) + 1
y_num = int(height/crop_height) + 1
max_x = width - crop_width
max_y = height - crop_height
for x in range(x_num):
    for y in range(y_num):
        start_x = min(x*crop_width, max_x)
        end_x = start_x + crop_width
        start_y = min(y*crop_height, max_y)
        end_y = start_y + crop_height
        test_image = tunnel_image[start_y:end_y, start_x:end_x, :]
        outputs = predictor(test_image)

        masks = outputs['instances'].pred_masks.cpu().numpy()
        # masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
        masks = masks[outputs['instances'].scores.cpu().numpy() > 0.5]
        mask = np.sum(masks, axis=0) > 0
        roi = test_image[mask]
        test_image[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
        tunnel_image[start_y:end_y, start_x:end_x, :] = test_image

save_path = tunnel_image_path.split('.png')[0] + '_seg.png'
print('Saved at', save_path)
cv2.imwrite(save_path, tunnel_image)

Saved at /home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/3yi_tunnel_seg.png


True